<div style="height:150px;background-repeat: no-repeat;background-color:#cccccc"><img src='https://sciencegateways.org/documents/33104/0/sgci-new-logo-words-below-black.png/2fd67b90-d490-4a61-8b4e-c525cfade141?t=1501001786898' style='height:150px'></div> 


In [8]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
import math
import time
from scgi_parameters import SCGI_UI
from scgi_utils import GetPbs as GetPbs, getStatus, isfloat, TemplatePBS
from scgi_connect import ShellHandler
from secrets import SECRETS
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import re
from SCGICatalog import SCGICatalogLocal
import os

In [2]:
instance_json = '''{ 
    "name" : "GaAs_Tutorial",
    "application": { "#ref" : "{app}#application" },
    "execution" : {
        "host" : { "#ref" : "{res}#host" },
        "connection" : { "#ref" : "{res}#((computeResources[?schedulerType==\'BATCH\'] | [0]).connections[?securityProtocol==\'PASSWORDS\'] | [0])" },
        "system" : { "#ref" : "{res}#(computeResources[?schedulerType==\'BATCH\'] | [0]).{jobManager:batchSystem.jobManager, commandPaths:batchSystem.commandPaths, partition:batchSystem.partitions[0]}"},
        "parallel" : { "#ref" : "{res}#((computeResources[?schedulerType==\'BATCH\'] | [0]).batchSystem.[executionCommands[?commandType==\'mpi\']][] | [0])" }
    },
    "storage":{
        "connection" : { "#ref" : "{res}#((computeResources[?schedulerType==\'BATCH\'] | [0]).connections[?securityProtocol==\'PASSWORDS\'] | [0])" },
        "filesystem" : { "#ref" : "{res}#(storageResources[?storageType==\'POSIX\'].fileSystems[] | [0])" } 
    }
}'''
qe_tacc = {
    "application":{
        "name" : "pw.x",
        "moduleDependencies" : ["qe/6.4.1"],
        "inputs" : { "PBS_TEMPLATE" : TemplatePBS() }
    }
}
qe_purdue = {
    "application":{
        "name" : "pw.x",
        "moduleDependencies" : ["quantum-espresso/6.3"],
        "inputs" : { "PBS_TEMPLATE" : TemplatePBS() }
    }
}
gh = SCGICatalogLocal()
params = [
    gh.derefResource(instance_json, {
        'res':gh.getResource('stampede2.tacc.xsede'), 
        'app':qe_tacc
    }),
    gh.derefResource(instance_json, {
        'res':gh.getResource('halstead.rcac.purdue'), 
        'app':qe_purdue
    }),
]
SCGI = {}


In [3]:
PBS = [p["application"]['inputs']['PBS_TEMPLATE'] for p in params]
TUTORIAL_NAME = [p["name"] for p in params]
APP = [p["application"]["name"] for p in params]
P_NAME = [p["execution"]["system"]["partition"]["name"] for p in params]
MPI = [p["execution"]['parallel']['commandPrefix'] for p in params]
modules = [(p["execution"]['parallel']['moduleDependencies'] + p["application"]["moduleDependencies"]) for p in params]
MODULES = [" \n".join(["module load " + m for m in mod]) for mod in modules]
PORT = [p["execution"]["connection"]['port'] for p in params]
HOST = [p["execution"]['host'] for p in params]
HOME = [p["storage"]["filesystem"]['homeDir'] for p in params]
COMMANDS = [{c['name']:c['path'] for c in  p["execution"]["system"]["commandPaths"]} for p in params]
SBATCH = [cd['SUBMISSION'] for cd in COMMANDS]
SESSIONS = [None, None]

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Calculating UV/Vis spectra on GaAs</h1></div>    

In this tutorial we will see how to calculate the band structures and the optical absorption spectra of
semiconductors. This tutorial we will perform calculations at the SCGI Resource, In order to connect to SCGI we need to use the `two-factor authentication' protocol.

For this you will need:
- The username
- The password
- The Dual Auth

Please fill out all the fields below and press the 'Connect' Button, if a connection is succesfull stablished the message "Connection created successfully" would be displayed.

If you get the message "ERROR: A new connection is required" on any of the steps below, you must update the information with a new google authenticator token, and press the button connect again.

Each time this notebook is loaded, this would assign a new working folder to store results. If you know the folder name containing previous results and skip some of the steps, please change it.

In [4]:
#SCGI_UI['s0']['folder'].value = "TEST1"
SCGI_UI['s_0']['user2'].value = SCGI_UI['s0']['user'].value.replace('denphi','dmejiapa')
SCGI_UI['s_0']['folder2'].value = re.sub('[0-9]*/denphi','dmejiapa',SCGI_UI['s_0']['folder1'].value)
SCGI_UI['s_0']['pwd2'].value = SECRETS['PID']
def CreateConnection( event ):    
    global SCGI_UI
    if True: #try
        SCGI_UI['s_0']['status'].value = "Connecting..."   
        SESSIONS[0] = ShellHandler( SCGI_UI['s_0']['code'].value, 
                                           SCGI_UI['s_0']['user1'].value, 
                                           SCGI_UI['s_0']['pwd1'].value, 
                                           HOST[0], 
                                           PORT[0])
        folder_name = HOME[0] + "/" + SCGI_UI['s_0']['folder1'].value
        if SESSIONS[0] is not None and SESSIONS[0].is_active():
            stdin, stdout, stderr, command = SESSIONS[0].execute("mkdir " + folder_name);
            stdin, stdout, stderr, command = SESSIONS[0].execute("cd " + folder_name);            
            SCGI_UI['s_0']['status'].value = "Connection created successfully"

        #except:
        #    SCGI_UI['s0']['status'].value = "ERROR: There has been an issue with the authentication"
        if True: #try
            SCGI_UI['s_0']['status'].value = "Connecting..."            
            SESSIONS[1] = ShellHandler( SCGI_UI['s_0']['code'].value, 
                                               SCGI_UI['s_0']['user2'].value, 
                                               SCGI_UI['s_0']['pwd2'].value, 
                                               HOST[1], 
                                               PORT[1])
            folder_name = HOME[1] + "/" + SCGI_UI['s_0']['folder2'].value
            if SESSIONS[1] is not None and SESSIONS[1].is_active():
                stdin, stdout, stderr, command = SESSIONS[1].execute("mkdir " + folder_name);
                stdin, stdout, stderr, command = SESSIONS[1].execute("cd " + folder_name);            
                SCGI_UI['s_0']['status'].value = "Connection created successfully"
            else:
                SCGI_UI['s_0']['status'].value = "ERROR: There has been an issue with the connection"     
            
        else:
            SCGI_UI['s_0']['status'].value = "ERROR: There has been an issue with the connection"            

SCGI_UI['s_0']['button']._click_handlers.callbacks = []
SCGI_UI['s_0']['button'].on_click(CreateConnection)
display(SCGI_UI['s_0']['display'])

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>Credentials<…

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Step 1: Crystal information</h1></div>

We consider GaAs this tutorial as a common semiconductor. We will start with a simple total energy calculation for GaAs in the diamond structure. In order to proceed we first need pseudopotentials, we need one pseudopotential for each atomic species (the pseudopotential describes the atomic nucleus and the electrons except the outermost (valence) shell). The QE pseudopotential libraries can be found at http://www.quantum-espresso.org/pseudopotentials , and will be downloaded in the working folder.

All the parameters in this input file have been optimized separately. the commands executed in the server are shown in the stdin tab, simulation output in the stdout and any error in the stderr tab. Press the "Calculate Self Consistency" button to start the simulation, the Job will be queued(PD), run(R) and then marked as (GC) when the job is complete

In [5]:
def CalculateSelfConsistency ( event ):
    global SCGI_UI
    if SESSIONS[0] is not None and SESSIONS[0].is_active():        
        id_name = TUTORIAL_NAME[0] + '_scf'
        pbs_file = id_name + ".pbs"
        SESSIONS[0].execute("wget http://www.quantum-espresso.org/upf_files/Ga.pz-bhs.UPF", SCGI_UI['s1']);
        SESSIONS[0].execute("wget http://www.quantum-espresso.org/upf_files/As.pz-bhs.UPF", SCGI_UI['s1']);
        inputdeck = SCGI_UI['s1']['input'].value.replace('\n', '\\n')
        SESSIONS[0].execute("printf \"" + inputdeck + "\" > " + id_name + ".in", SCGI_UI['s1']);
        SESSIONS[0].execute("printf \"" + GetPbs(PBS[0], id_name, APP[0], P_NAME[0], MPI[0], MODULES[0]).replace('\n', '\\n') + "\" > " + pbs_file, SCGI_UI['s1']);
        stdin, stdout, stderr, command = SESSIONS[0].execute(SBATCH[0] + " " + pbs_file, SCGI_UI['s1']);
        try:
            code = stdout[len(stdout)-1].strip('\n')
            code = code.split(' ')
            code = code[len(code)-1]
            if (code.isdigit()):
                SCGI_UI['s1']['job_id'].value = code
                status = 'Q'
                while status != 'CG' and status != 'X' and status != 'ST' and status != 'O' :
                    status, response, command = getStatus( SESSIONS[0], SCGI_UI['s1']['job_id'].value, SCGI_UI['s1'] )
                    SCGI_UI['s1']['status'].value = status
                    time.sleep(5)                    
                SESSIONS[0].execute("cat " + id_name + ".out", SCGI_UI['s1']);
                folder_name = HOME[0] + "/" + SCGI_UI['s_0']['folder1'].value
                SESSIONS[0].scp.get(folder_name, recursive=True)
            else : 
                SCGI_UI['s1']['job_id'].value = "Error with the PBS submittion"
        except:
            SCGI_UI['s1']['job_id'].value = "Error with the PBS submittion"
    else :
        print ("ERROR: A new connection is required")

SCGI_UI['s1']['button']._click_handlers.callbacks = []
SCGI_UI['s1']['button'].on_click(CalculateSelfConsistency)
display (SCGI_UI['s1']['display'])



<div style="height:50px;background-repeat: no-repeat;background-color:#cccccc"><img src='https://www.tacc.utexas.edu/tacc-new-theme/images/logo.svg' style='position:absolute;right:20px;height:50px'><h1 style="text-align:left;padding-top:10px">Stampede 2 (Texas Advanced Computing Center)</h1></div> 

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Step 2: Calculate Band Structures</h1></div>

Then, we want to calculate the band structure. This calculation is non self-consistent, in the sense that we use values for the ground-state electron density, Hartree, and exchange and correlation potentials. In a non self-consistent calculation the code pw.x determines the Kohn-Sham eigenfunctions and eigenvalues without upgrading the Kohn-Sham Hamiltonian at every step. This is achieved by using the keyword calculation = 'bands' and by specifying the k-points for which we want the eigenvalues:

In this input file the keyword tpiba_b after K_POINTS specifies that we want pw.x to generate a path going through the points specified in the list. The following number (3) is the number of vertices, and the integer following the coordinates (20) is the number of points in each segment. So in this case we will have 20 points from $L = (1/2,1/2,1/2)2\pi/a$ to $\Gamma$ = (0,0,0) and 20 points from $\Gamma=(0,0,0)$ to $X=(1,0,0)2\pi/a$. The points are given in Cartesian coordinates and in units of $2\pi/a$. In this input file we also specify the number of bands that we want to calculate, we are setting nbnd = 8.

Press the "Calculate Bandstructure" button to start the simulation, the Job will be queued(Q), run(R) and then marked as (C) when the job is complete

In [9]:
def CalculateBandStructure ( event ):
    global SCGI_UI
    if SESSIONS[1] is not None and SESSIONS[1].is_active():   
        folder_name = HOME[1] + "/" + SCGI_UI['s_0']['folder2'].value
        for filename in os.scandir(re.sub('[0-9]*/denphi','.',SCGI_UI['s_0']['folder1'].value)):
            SESSIONS[1].scp.put(filename.path, folder_name, recursive=True)        
        id_name = TUTORIAL_NAME[1] + '_bands'
        pbs_file = id_name + ".pbs"
        inputdeck = SCGI_UI['s2']['input'].value.replace('\n', '\\n')
        SESSIONS[1].execute("printf \"" + inputdeck + "\" > " + id_name + ".in", SCGI_UI['s2']);
        SESSIONS[1].execute("printf \"" + GetPbs(PBS[1], id_name, APP[1], P_NAME[1], MPI[1], MODULES[1]).replace('\n', '\\n') + "\" > " + pbs_file, SCGI_UI['s1']);
        stdin, stdout, stderr, command = SESSIONS[1].execute(SBATCH[1] + " " + pbs_file, SCGI_UI['s2']);
        try:
            code = stdout[len(stdout)-1].strip('\n')
            code = code.split(' ')
            code = code[len(code)-1]
            if (code.isdigit()):
                SCGI_UI['s2']['job_id'].value = code
                status = 'Q'
                while status != 'CG' and status != 'X' and status != 'ST' and status != 'O' :
                    status, response, command = getStatus( SESSIONS[1], SCGI_UI['s2']['job_id'].value, SCGI_UI['s2'] )
                    SCGI_UI['s2']['status'].value = status
                    time.sleep(5)
                stdin, stdout, stderr, command = SESSIONS[1].execute("cat " + id_name + ".out", SCGI_UI['s2']);
            else : 
                SCGI_UI['s2']['job_id'].value = "Error with the PBS submittion"
        except:
            SCGI_UI['s2']['job_id'].value = "Error with the PBS submittion"
    else :
        print ("ERROR: A new connection is required")

    
SCGI_UI['s2']['button']._click_handlers.callbacks = []
SCGI_UI['s2']['button'].on_click(CalculateBandStructure)
display (SCGI_UI['s2']['display'])


<div style="height:50px;background-repeat: no-repeat;background-color:#cccccc"><img src='https://engineering.purdue.edu/Wraps/ECO/wrap7/images/logo.png' style='position:absolute;right:20px;height:50px'><h1 style="text-align:left;padding-top:10px">Halstead (Purdue Community Cluster)</h1></div> 

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><img src='HubzeroLogo.png' style='position:absolute;right:20px;width:110px;height:32px'><h1 style="text-align:center;padding-top:3px">Step 3: Extract Bandstructures</h1></div>

For each k-point in the input file, we have the coordinates of the point and the calculated eigenvalues in eV, in this case we requested 8 bands. In order to plot the bands along the chosen path, we must extract these eigenvalues, and calculate the distance covered as we move along the path $L \longrightarrow \Gamma \longrightarrow X$.

* ExtractKPoints: Extract all k-points and their respective eigenvalues
* CalculatePath: Calculates the distance of each k_point in the path
* CreateDataPlot: Creates scattered lines along each band
* CreateAdditionalTics: Includes labels for known symmetry points.

Press the "Extract Bandstructure" button to start the processing, the bandstructures will be visualized using the plotly library

In [7]:
def ExtractKPoints ( stdout ):        
    l = len(stdout)
    i=0
    k_points = []
    while i<l:
        if ' k = ' in stdout[i]: #new k point found
            point = {}
            point['k'] = [float(s) for s in stdout[i].split() if isfloat(s)]
            i = i+2
            point['v'] = []
            while (i<l and stdout[i].strip() != ""):
                point['v'] = point['v'] + [float(s) for s in stdout[i].split() if isfloat(s)]
                i = i + 1
            point['total'] = len(point['v'])
            k_points.append(point)
        i = i+1
    return k_points

def CalculatePath ( k_points ):        
    path_len = 0
    for i in range(len(k_points)):
        if i > 0:
            p1= k_points[i-1]['k']
            p2= k_points[i]['k']
            path_len = path_len + math.sqrt(math.pow(p2[0]-p1[0],2)+math.pow(p2[1]-p1[1],2)+math.pow(p2[2]-p1[2],2))                                                
        k_points[i]['p'] = path_len
    x_points = [k['p'] for k in k_points]
    return k_points, x_points

def CreateDataPlot( k_points, x_points, ezero, total_bands ):
    data = []
    emin = 0
    emax = 0
    for k in range(total_bands):
        y_points = [p['v'][k]-ezero for p in k_points]
        emax = max(emax, max(y_points))
        emin = min(emin, min(y_points))
        if all(v > 0 for v in y_points):
            band_color = 'rgb(205, 12, 24)'
        else:
            band_color = 'rgb(22, 96, 167)'
        trace = go.Scatter(
            x = x_points,
            y = y_points,
            mode = 'lines',
            line = dict(color = (band_color)))
        data.append(trace)
    return data, emax, emin

def CreateAdditionalTics( data, k_points, labels, emax, emin ):
    ticktext=[]
    tickvals=[]
    for key, value in labels.items():                    
        step = 0
        for k in k_points:
            if (k['k'][0]==value[0] and k['k'][1]==value[1] and k['k'][2]==value[2]):
                step = k['p']
        trace = go.Scatter(
            x=[step, step],
            y=[emin, emax],
            mode="lines",
            line=dict(color="#111111", width=1),
            showlegend=False)
        data.append(trace)
        ticktext.append(key)
        tickvals.append(step)
    return data, ticktext, tickvals



def VisualizeBandStructure ( event ):        
    global SCGI_UI
    if SESSIONS[1] is not None and SESSIONS[1].is_active():
        labels = {"L":[0.5,0.5,0.5], "G":[0.0, 0.0, 0.0], "X":[1.0, 0.0, 0.0]}
        ezero = 6.2057  
        SCGI_UI['s3']['output'].clear_output()
        with SCGI_UI['s3']['output']:            
            id_name = TUTORIAL_NAME[1] + '_bands'
            stdin, stdout, stderr, command = SESSIONS[1].execute("cat " + id_name + ".out");
            k_points = ExtractKPoints(stdout)
            SCGI_UI['s3']['input'].value = '\n'.join([str(c) + ' ' + str(k['v']) for c,k in enumerate(k_points)])
            k_points, x_points = CalculatePath(k_points)
            data, emax, emin = CreateDataPlot(k_points, x_points, ezero, k_points[0]['total'])
            data, ttext, tvals = CreateAdditionalTics(data, k_points, labels, emax, emin)
            layout = go.Layout(title='GaAs Bandstructure',
                    xaxis=dict(title = 'k-point path [2pi/a]', autorange=True, exponentformat = "e", ticktext=ttext,tickvals=tvals),
                    yaxis=dict(title = 'Energy (ev)', autorange=False, exponentformat = "e", range=[emin, emax],),
                    showlegend=False)                    
            fig = go.Figure(data=data, layout=layout)
            iplot(fig, show_link=False)

    else :
        print ("ERROR: A new connection is required")
        
SCGI_UI['s3']['button']._click_handlers.callbacks = []
SCGI_UI['s3']['button'].on_click(VisualizeBandStructure)
display(SCGI_UI['s3']['display'])


Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>band-structu…

<div style="height:50px;background-repeat: no-repeat;background-color:#cccccc"><img src='https://nanohub.org/app/site/media/branding/nanoHUB_logo_color.png' style='position:absolute;right:20px;height:50px'><h1 style="text-align:left;padding-top:10px">nanoHUB (Jupyter - Plotly Widget)</h1></div> 